In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install seaborn

In [5]:
import seaborn as sns

In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [7]:
import runml.pipeline
from runml import pipeline,findata


In [8]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [ ]:
!git clone https://github.com/ahsank/runml

Run following if the library code is modified locally

In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [8]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
model = "gstock-6b-test"
tickers = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-6b-test"
tickers = ['AAL', 'ALB', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-6b-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-7a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [9]:
findata.EPOCHS=200
model = "mcap-7a-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)


In [11]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
30,PYPL,63.71,0.93,1101.38,1347.49,72.10,79.000000,0.10,0.64,805.97,958.29,85.650002,0.19,0.75,987.56,1575.67,65.629997,-0.09,0.10,8.600000
14,DE,111.43,0.87,1409.88,1364.31,414.61,372.390015,-0.10,0.79,2264.14,1563.24,428.350006,0.03,0.65,1537.97,759.45,375.000000,-0.10,-0.10,7.400000
7,BKNG,1192.62,0.87,11701.18,10526.84,2866.92,2589.929932,-0.10,0.78,12465.46,10161.94,2808.969971,-0.02,0.65,9462.37,9644.61,2610.540039,-0.09,-0.09,7.255556
27,NVDA,42.11,0.94,2021.05,1107.57,454.69,423.010010,-0.07,0.75,2357.02,718.77,460.089996,0.01,0.63,1436.20,111.80,383.100006,-0.16,-0.07,8.542857
0,AAPL,49.28,0.93,868.74,512.27,190.69,173.860001,-0.09,0.84,1054.58,515.40,189.490005,-0.01,0.63,524.54,185.96,176.990005,-0.07,-0.07,8.300000
16,DIS,86.23,0.87,693.58,632.42,88.62,95.290001,0.08,0.60,380.36,380.58,96.110001,0.08,0.66,460.37,503.29,84.320000,-0.05,0.08,7.075000
25,MSFT,130.76,0.93,1559.00,859.51,345.24,324.399994,-0.06,0.81,1527.05,829.73,337.869995,-0.02,0.65,1283.19,686.27,312.149994,-0.10,-0.06,8.133333
26,NKE,63.08,0.90,599.51,689.86,107.95,113.910004,0.06,0.74,487.70,487.39,117.199997,0.09,0.65,532.88,488.06,100.440002,-0.07,0.06,7.333333
24,MMC,74.67,0.89,673.10,355.51,186.72,170.750000,-0.09,0.77,625.46,258.65,188.279999,0.01,0.62,616.83,201.97,174.869995,-0.06,-0.06,7.066667
20,INTC,25.09,0.94,223.73,245.90,33.15,34.770000,0.05,0.73,251.98,220.70,38.849998,0.17,0.71,216.47,251.15,32.560001,-0.02,0.05,8.200000


In [10]:
findata.EPOCHS=200
model = "vols-6a-test"
tickers = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-6a-test"
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-5b-test"
tickers = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-5b-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)

In [12]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
41,RIDE,15.03,0.67,138.66,1186.76,2.62,26.280001,9.03,0.42,200.05,1147.10,10.500000,3.01,0.73,183.41,1336.24,3.78,0.44,3.01,6.590365
1,APRN,9.87,0.86,1308.76,1607.94,5.18,11.500000,1.22,0.53,847.28,1076.95,9.330000,0.80,0.75,735.88,1313.02,5.25,0.01,0.80,8.425000
13,CZOO,4.66,0.71,211.79,793.32,1.49,11.000000,6.38,0.42,-272.63,475.93,2.870000,0.93,0.75,245.54,887.29,0.59,-0.61,0.93,6.788172
33,NSTG,5.07,0.83,327.52,440.70,3.92,8.140000,1.08,0.69,413.97,408.46,6.610000,0.69,0.69,309.36,334.22,3.96,0.01,0.69,8.053623
34,OMQS,3.38,0.88,62.93,78.13,3.81,6.050000,0.59,0.66,88.00,74.13,6.440000,0.69,0.78,94.52,98.16,3.75,-0.02,0.59,8.596610
38,OUST,6.33,0.83,304.25,651.35,6.08,21.559999,2.55,0.53,147.32,432.39,9.710000,0.60,0.71,132.07,572.86,6.06,-0.00,0.60,8.016667
40,REAL,2.46,0.73,74.77,122.16,2.39,6.410000,1.68,0.63,30.45,80.91,3.870000,0.62,0.72,83.19,140.34,2.38,-0.00,0.62,6.864516
36,OPEN,1.67,0.88,124.35,178.06,4.44,12.530000,1.82,0.72,158.88,169.09,6.050000,0.36,0.79,84.85,197.85,4.96,0.12,0.36,8.466667
46,SFT,3.32,0.80,395.10,503.51,2.24,7.320000,2.27,0.48,297.43,314.99,3.140000,0.40,0.61,181.91,482.95,1.33,-0.41,0.40,7.500000
45,STEM,4.74,0.89,231.07,235.07,6.59,9.850000,0.49,0.61,-7.05,115.48,8.820000,0.34,0.78,225.96,330.25,5.59,-0.15,0.34,8.576471


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [12]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
mcap-7a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,222,185 bytes  received 35 bytes  6,444,440.00 bytes/sec
total size is 205,595,784  speedup is 63.81
